## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## Read cleaned data files

### Read cleaned jobs listings file and save in dataframe

In [2]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

### Check columns info in jobs listings dataframe

In [3]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3690 entries, 0 to 3689
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3690 non-null   object 
 1   salary_estimate          3690 non-null   object 
 2   job_description          3690 non-null   object 
 3   rating                   3690 non-null   float64
 4   company_name             3690 non-null   object 
 5   location                 3690 non-null   object 
 6   headquarters             3690 non-null   object 
 7   size                     3690 non-null   object 
 8   founded                  3690 non-null   int64  
 9   type_of_ownership        3690 non-null   object 
 10  industry                 3690 non-null   object 
 11  sector                   3690 non-null   object 
 12  revenue                  3690 non-null   object 
 13  competitors              3690 non-null   object 
 14  easy_apply              

## Data Preprocessing

### Define stopwords to append to default stopwords

In [4]:
# append special stopwords in articles to default and common stopwords
new_stopwords = STOPWORDS.union(set(['job', 'skill', 'experience', 'team', 'data', 'use', 'like', 'business', 'work', 'ability', 'let', 'example'
                'need', 'new', 'user', 'opportunity', 'candidate', 'provide', 'company', 'one', 'used', 'need', 
                'see', 'make', 'follow', 'going', 'will', 'want', 'well', 'find', 'give', 'change', 'look', 'first', 'using',
                'know', 'model', 'science', 'think', 'year', 'years', 'looking', 'machine', 'learning', 'job', 'description']))

### Method to preprocess data for articles

In [5]:
# lemmatize = words change to noun
# stemmed = words reduced to root form
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

# Convert a document into a list of tokens.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in new_stopwords:
            result.append(lemmatize(token))
    return result

In [6]:
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [7]:
# implements the concept of a Dictionary – a mapping between words and their integer ids.
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [8]:
# Convert into the bag-of-words (BoW) format
jobs_listings_bow_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

In [9]:
# Convert into TF-IDF format
jobs_listings_tfidf = models.TfidfModel(jobs_listings_bow_corpus)
jobs_listings_tfidf_corpus = jobs_listings_tfidf[jobs_listings_bow_corpus]

## Common method to find dominant topics in the jobs description

In [10]:
def get_dominant_topics(model, corpus):
    dominant_topics_df = pd.DataFrame()
    for i, row in enumerate(model[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # => dominant topic
                wp = model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                dominant_topics_df = dominant_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    dominant_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    dominant_topics_df["Text"] = cleaned_jobs_listings_df['cleaned_job_description']
    dominant_topics_df = dominant_topics_df.sort_values(by=['Perc_Contribution'], ascending=False)
    return dominant_topics_df.head(10)

## LSA Model

### Result using LSA model + Bag of words

In [11]:
jobs_listings_bow_lsamodel = LsiModel(jobs_listings_bow_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [12]:
jobs_listings_bow_lsamodel.print_topics(num_words=7)

[(0,
  '0.176*"development" + 0.166*"analysis" + 0.153*"skill" + 0.144*"product" + 0.141*"technology" + 0.139*"analytics" + 0.137*"information"'),
 (1,
  '-0.253*"analytics" + 0.203*"information" + 0.200*"research" + 0.177*"employee" + 0.175*"laboratory" + -0.173*"product" + 0.163*"required"'),
 (2,
  '-0.301*"research" + 0.273*"information" + 0.212*"application" + -0.192*"laboratory" + -0.186*"product" + -0.175*"clinical" + -0.164*"scientist"'),
 (3,
  '0.229*"product" + 0.201*"status" + -0.198*"management" + 0.169*"people" + -0.158*"process" + 0.156*"gender" + -0.155*"project"'),
 (4,
  '0.389*"development" + -0.325*"analysis" + -0.284*"analytics" + 0.225*"technology" + -0.191*"statistical" + -0.140*"insight" + 0.140*"design"'),
 (5,
  '-0.485*"research" + 0.326*"laboratory" + 0.213*"medical" + 0.195*"service" + -0.158*"analysis" + 0.154*"patient" + -0.153*"development"'),
 (6,
  '-0.427*"product" + 0.364*"research" + -0.222*"process" + 0.156*"solution" + 0.153*"technology" + 0.150*"

In [13]:
get_dominant_topics(jobs_listings_bow_lsamodel, jobs_listings_bow_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
1889,0.0,39.6849,"development, analysis, skill, product, technol...",job descriptionjob title data analystsalary 18...
591,0.0,39.4304,"development, analysis, skill, product, technol...",job descriptionoperation analytics manager tr...
891,0.0,32.8127,"development, analysis, skill, product, technol...",at west monroe our people are our businesswe p...
788,0.0,31.5336,"development, analysis, skill, product, technol...",address111 w monroe 115 s lasallejob family g...
1485,0.0,31.4038,"development, analysis, skill, product, technol...",descriptionthe lead of cybersecurity data prot...
770,0.0,28.3336,"development, analysis, skill, product, technol...",address111 w monroe 115 s lasallejob family g...
874,0.0,28.1692,"development, analysis, skill, product, technol...",please make sure to read the job posting in it...
471,0.0,27.2988,"development, analysis, skill, product, technol...",position descriptionposition title senior data...
97,0.0,27.1588,"development, analysis, skill, product, technol...",search by keywordsearch by locationclearsend m...
1199,5.0,27.0629,"research, laboratory, medical, service, analys...",work shift eveningwork week m fjob summary082...


### Result using LSA model + TF-IDF

In [14]:
jobs_listings_tfidf_lsamodel = LsiModel(jobs_listings_tfidf_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [15]:
jobs_listings_tfidf_lsamodel.print_topics(num_words=7)

[(0,
  '0.096*"analytics" + 0.087*"product" + 0.077*"research" + 0.077*"solution" + 0.076*"customer" + 0.073*"model" + 0.073*"client"'),
 (1,
  '0.261*"laboratory" + 0.172*"clinical" + 0.158*"cell" + -0.114*"aws" + 0.114*"scientific" + 0.114*"assay" + 0.111*"biology"'),
 (2,
  '-0.181*"analyze" + -0.175*"locate" + -0.168*"printout" + -0.163*"databasesdata" + -0.149*"qualityacquire" + -0.149*"setsfilter" + -0.149*"disseminate"'),
 (3,
  '-0.657*"tutor" + -0.441*"tutoring" + -0.163*"student" + -0.117*"choose" + -0.112*"session" + -0.088*"university" + 0.086*"cell"'),
 (4,
  '-0.330*"tutor" + -0.221*"tutoring" + 0.161*"university" + 0.130*"clery" + 0.127*"security" + -0.120*"cell" + 0.117*"investigation"'),
 (5,
  '0.171*"aws" + 0.160*"laboratory" + 0.146*"pipeline" + 0.143*"cell" + 0.136*"cloud" + 0.122*"spark" + 0.107*"etl"'),
 (6,
  '-0.233*"cell" + -0.137*"biology" + 0.135*"laboratory" + 0.125*"hospital" + -0.118*"assay" + 0.109*"care" + 0.100*"medical"'),
 (7,
  '0.253*"laboratory" +

In [16]:
get_dominant_topics(jobs_listings_tfidf_lsamodel, jobs_listings_tfidf_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
2585,9.0,0.8526,"bull, status, gender, employment, protected, p...",job description bull analyze and understand da...
2586,9.0,0.8483,"bull, status, gender, employment, protected, p...",bull analyze and understand data sources apis ...
2521,9.0,0.8472,"bull, status, gender, employment, protected, p...",role developer data engineer location irving ...
3496,9.0,0.8343,"bull, status, gender, employment, protected, p...",job description analyze and understand data so...
1488,9.0,0.8144,"bull, status, gender, employment, protected, p...",description the data engineer will be primaril...
1747,9.0,0.8041,"bull, status, gender, employment, protected, p...",data analyst horsham pa 19044 my client a loa...
3520,9.0,0.7910,"bull, status, gender, employment, protected, p...",we are in critical need of senior bigdata engi...
2923,9.0,0.7888,"bull, status, gender, employment, protected, p...",skilljob requirements bull bachelorrsquos degr...
932,9.0,0.7794,"bull, status, gender, employment, protected, p...",title provider analytics data analyst mandator...
2327,9.0,0.7654,"bull, status, gender, employment, protected, p...",client seeks a talented resultsoriented data s...


## LDA Model

### Result using LDA model + Bag of words

In [17]:
jobs_listings_bow_ldamodel = gensim.models.LdaMulticore(jobs_listings_bow_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [18]:
jobs_listings_bow_ldamodel.print_topics(num_words=7)

[(0,
  '0.005*"research" + 0.005*"analysis" + 0.005*"skill" + 0.004*"development" + 0.004*"solution" + 0.004*"support" + 0.004*"technology"'),
 (1,
  '0.007*"skill" + 0.006*"product" + 0.005*"development" + 0.004*"analysis" + 0.004*"including" + 0.004*"analytics" + 0.004*"customer"'),
 (2,
  '0.006*"analytics" + 0.006*"analysis" + 0.005*"development" + 0.005*"skill" + 0.005*"technology" + 0.004*"product" + 0.004*"tool"'),
 (3,
  '0.006*"product" + 0.006*"technology" + 0.005*"research" + 0.005*"analytics" + 0.005*"development" + 0.005*"knowledge" + 0.004*"project"'),
 (4,
  '0.006*"development" + 0.006*"product" + 0.006*"process" + 0.005*"project" + 0.005*"technology" + 0.005*"technical" + 0.004*"analytics"'),
 (5,
  '0.005*"skill" + 0.005*"status" + 0.005*"development" + 0.005*"technology" + 0.005*"service" + 0.004*"information" + 0.004*"product"'),
 (6,
  '0.008*"analysis" + 0.005*"development" + 0.005*"skill" + 0.005*"information" + 0.005*"including" + 0.004*"working" + 0.004*"soluti

In [19]:
pyLDAvis.gensim_models.prepare(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.003044 -0.006070       1        1  17.960567
7      0.008646 -0.001938       2        1  17.817106
6     -0.001255  0.007028       3        1  12.766118
0      0.000246 -0.001490       4        1  12.231046
3      0.003776 -0.001792       5        1   9.837215
1      0.005514  0.009117       6        1   9.506473
8      0.000944  0.000015       7        1   6.802009
4      0.001930 -0.001383       8        1   6.799841
5     -0.014223  0.004727       9        1   4.588616
9     -0.008623 -0.008212      10        1   1.691010, topic_info=            Term         Freq        Total Category  logprob  loglift
50   development  4794.000000  4794.000000  Default  30.0000  30.0000
305   technology  3943.000000  3943.000000  Default  29.0000  29.0000
132      product  4200.000000  4200.000000  Default  28.0000  28.0000
426      process  3124.000000  3124.000000  Default  27.0000  27.0000
133      project  3512.000000  3512.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
557       system    40.144159  2551.976521  Topic10  -5.9661  -0.0723
142     research    46.508154  3565.091355  Topic10  -5.8190  -0.2595
39      customer    40.539502  2721.968073  Topic10  -5.9563  -0.1270
177         tool    41.804023  3216.998894  Topic10  -5.9256  -0.2634
197      working    40.413224  3588.402904  Topic10  -5.9595  -0.4065

[848 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6210       1  0.136597    accenture
6210       2  0.136597    accenture
6210       3  0.045532    accenture
6210       4  0.455324    accenture
6210       5  0.045532    accenture
...      ...       ...          ...
46116      3  0.715813  xanthomonas
7638       1  0.220524  âexperience
7638       2  0.661572  âexperience
7641       1  0.219912      âshould
7641       2  0.659737      âshould

[2151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 7, 1, 4, 2, 9, 5, 6, 10])

In [20]:
get_dominant_topics(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
1141,8.0,0.9986,"technology, skill, solution, analysis, service...",work shift daywork week variesjob summarythe m...
1973,2.0,0.9985,"analytics, analysis, development, skill, techn...",senior social scientistleonie is currently see...
1239,8.0,0.9985,"technology, skill, solution, analysis, service...",work shift nightwork week m fjob summaryfull ...
1234,8.0,0.9985,"technology, skill, solution, analysis, service...",work shift daywork week m f w occasional week...
1233,8.0,0.9985,"technology, skill, solution, analysis, service...",work shift daywork week m f w occasional week...
1354,0.0,0.9985,"research, analysis, skill, development, soluti...",who we arefueled by a fundamental belief that ...
1599,6.0,0.9984,"analysis, development, skill, information, inc...",university overviewthe the largest private em...
1222,8.0,0.9984,"technology, skill, solution, analysis, service...",work shift nightwork week variesjob summaryjob...
1202,8.0,0.9984,"technology, skill, solution, analysis, service...",work shift daywork week variesjob summarythe m...
1197,8.0,0.9984,"technology, skill, solution, analysis, service...",work shift thirdwork week variesjob summary082...


### Result using LDA model + TF-IDF

In [21]:
jobs_listings_tfidf_ldamodel = gensim.models.LdaMulticore(jobs_listings_tfidf_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [22]:
jobs_listings_tfidf_ldamodel.print_topics(num_words=7)

[(0,
  '0.001*"research" + 0.001*"laboratory" + 0.001*"client" + 0.001*"health" + 0.001*"clinical" + 0.001*"employment" + 0.001*"bull"'),
 (1,
  '0.001*"analytics" + 0.001*"model" + 0.001*"statistical" + 0.001*"product" + 0.001*"customer" + 0.001*"management" + 0.001*"research"'),
 (2,
  '0.001*"product" + 0.001*"analytics" + 0.001*"customer" + 0.001*"solution" + 0.001*"model" + 0.001*"software" + 0.001*"statistical"'),
 (3,
  '0.001*"product" + 0.001*"analytics" + 0.001*"customer" + 0.001*"research" + 0.001*"solution" + 0.001*"engineering" + 0.001*"marketing"'),
 (4,
  '0.001*"product" + 0.001*"analytics" + 0.001*"client" + 0.001*"solution" + 0.001*"insight" + 0.001*"research" + 0.001*"health"'),
 (5,
  '0.001*"analytics" + 0.001*"client" + 0.001*"research" + 0.001*"aws" + 0.001*"tutor" + 0.001*"product" + 0.001*"status"'),
 (6,
  '0.001*"analytics" + 0.001*"clinical" + 0.001*"laboratory" + 0.001*"research" + 0.001*"product" + 0.001*"model" + 0.001*"algorithm"'),
 (7,
  '0.001*"analyt

In [23]:
pyLDAvis.gensim_models.prepare(jobs_listings_tfidf_ldamodel, jobs_listings_tfidf_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.005994 -0.018642       1        1  16.776877
9      0.009980  0.005227       2        1  12.007274
7     -0.002609 -0.007539       3        1  11.499727
3      0.013612  0.007330       4        1  11.147313
0      0.000473  0.000259       5        1   9.176175
4      0.001126  0.005389       6        1   8.976706
2     -0.003351 -0.000573       7        1   8.782460
8     -0.005819  0.005926       8        1   7.711460
1     -0.007455 -0.000476       9        1   7.359013
5     -0.011953  0.003099      10        1   6.562996, topic_info=              Term      Freq      Total Category  logprob  loglift
13281        tutor  6.000000   6.000000  Default  30.0000  30.0000
20615         bull  7.000000   7.000000  Default  29.0000  29.0000
13282     tutoring  4.000000   4.000000  Default  28.0000  28.0000
8536       quantum  4.000000   4.000000  Default  27.0000  27.0000
18559       middot  2.000000   2.000000  Default  26.0000  26.0000
...            ...       ...        ...      ...      ...      ...
259      knowledge  1.077490  17.389094  Topic10  -7.7679  -0.0575
444       solution  1.138738  21.607449  Topic10  -7.7126  -0.2194
403     management  1.102258  19.256100  Topic10  -7.7451  -0.1367
41        database  1.108498  20.062536  Topic10  -7.7395  -0.1721
50     development  1.094639  20.106752  Topic10  -7.7521  -0.1869

[800 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6210       1  0.520652    accenture
11916      3  0.406654         acid
564        1  0.199001  advertising
564        2  0.199001  advertising
564        3  0.199001  advertising
...      ...       ...          ...
322        8  0.083866        youll
322        9  0.083866        youll
322       10  0.083866        youll
9186       2  0.393309           yr
9186       6  0.393309           yr

[1260 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 8, 4, 1, 5, 3, 9, 2, 6])

In [24]:
get_dominant_topics(jobs_listings_tfidf_ldamodel, jobs_listings_tfidf_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
3359,4.0,0.9502,"product, analytics, client, solution, insight,...",we are hiringdata engineer fulltime day aust...
3261,4.0,0.9495,"product, analytics, client, solution, insight,...",why join technologies leverages technology to...
1141,6.0,0.9494,"analytics, clinical, laboratory, research, pro...",work shift daywork week variesjob summarythe m...
3287,6.0,0.9493,"analytics, clinical, laboratory, research, pro...",location pickle research campusjob posting tit...
1234,6.0,0.9492,"analytics, clinical, laboratory, research, pro...",work shift daywork week m f w occasional week...
3063,6.0,0.9490,"analytics, clinical, laboratory, research, pro...",about usat we have our eyes set on an ambitio...
3325,4.0,0.9490,"product, analytics, client, solution, insight,...",we are hiringdata engineer fulltime day aust...
289,6.0,0.9487,"analytics, clinical, laboratory, research, pro...",senior data scientist description the senior d...
1199,6.0,0.9486,"analytics, clinical, laboratory, research, pro...",work shift eveningwork week m fjob summary082...
1233,6.0,0.9483,"analytics, clinical, laboratory, research, pro...",work shift daywork week m f w occasional week...
